## AgentCore Evaluations - Strands エージェントのオンライン評価

このチュートリアルでは、Strands エージェントに適用される AgentCore Evaluations のオンライン評価の使用方法を学びます。

このラボを実行するには、まず [00-prereqs](../../00-prereqs) フォルダのコードを使用して Strands エージェントを作成し、[01-creating-custom-evaluators](../../01-creating-custom-evaluators) のコードを使用してカスタム評価メトリクスを作成しておく必要があります

### 学習内容
- AgentCore Starter toolkit を使用してトレースにオンライン評価を実行する方法

### チュートリアル詳細

| 情報               | 詳細                                                                          |
|:-------------------|:------------------------------------------------------------------------------|
| チュートリアルタイプ | オンライン評価メトリクス（ビルトインおよびカスタム）を使用した Strands エージェントの評価 |
| チュートリアル構成   | ビルトインおよびカスタム評価メトリクスでの自動評価設定                          |
| チュートリアル分野   | 横断的                                                                        |
| 複雑さ              | 簡単                                                                          |
| 使用 SDK            | Amazon Bedrock AgentCore Starter toolkit                                      |

### オンライン評価

オンライン評価は、デプロイされたエージェントのライブトラフィック品質監視を可能にします。特定の選択されたインタラクションを分析するオンデマンド評価とは異なり、オンライン評価はリアルタイムトラフィックに基づいて本番環境でのエージェントのパフォーマンスを継続的に評価します。

オンライン評価は3つの主要なコンポーネントで構成されています。まず、**セッションサンプリングとフィルタリング**により、エージェントインタラクションを評価するための特定のルールを設定できます。すべてのセッションの一部を評価するパーセンテージベースのサンプリング（例：10%）を設定したり、より絞り込んだ評価のための条件フィルタを定義したりできます。次に、新しいカスタム評価メトリクスの作成、既存のカスタム評価メトリクスの使用、またはビルトイン評価メトリクスからの選択を含む**複数の評価方法**から選択できます。最後に、**監視と分析**機能により、ダッシュボードで集計スコアを表示し、時間の経過に伴う品質トレンドを追跡し、低スコアのセッションを調査し、入力から出力までの完全なインタラクションフローを分析できます。

オンライン評価では、特定のデータソース（エージェントトレースを含む CloudWatch ロググループまたは AgentCore Runtime エンドポイント）を自動的に監視するようにシステムを設定します。サービスはサンプリングとフィルタリングルールに基づいて受信トレースを継続的に処理し、選択した評価メトリクスをリアルタイムで適用し、分析用の詳細な結果を CloudWatch に出力します。この評価タイプは、本番監視、品質低下の早期検出、ユーザーインタラクションのパターン特定、大規模での一貫したエージェントパフォーマンスの維持に特に便利です。

オンライン評価設定を作成して有効にすると、サービスはバックグラウンドで継続的に実行され、セッションが発生するとそれを評価し、エージェントの品質メトリクスへの継続的な可視性を提供します。評価戦略を変更するニーズに合わせて、いつでも設定を一時停止、変更、または削除できます。

### AgentCore Observability でエージェントからトレースを生成

AgentCore Observability は、[OpenTelemetry (OTEL)](https://opentelemetry.io/) トレースを活用して詳細な実行データをキャプチャし構造化することで、呼び出し中のエージェントの動作を包括的に可視化します。AgentCore は [AWS Distro for OpenTelemetry (ADOT)](https://aws-otel.github.io/) を利用して、さまざまなエージェントフレームワーク間で異なるタイプの OTEL トレースを計装します。

エージェントが AgentCore Runtime でホストされている場合（このチュートリアルのエージェントのように）、AgentCore Observability の計装は最小限の設定で自動的に行われます。必要なのは `requirements.txt` に `aws-opentelemetry-distro` を含めることだけで、AgentCore Runtime が OTEL 設定を自動的に処理します。エージェントが AgentCore Runtime で実行されていない場合は、AgentCore Observability で利用できるようにするために ADOT で計装する必要があります。テレメトリデータを CloudWatch に送信するための環境変数を設定し、OpenTelemetry 計装でエージェントを実行する必要があります。

プロセスは以下のようになります:

![session_traces](../../images/observability_traces.png)

セッショントレースが AgentCore Observability で利用可能になると、AgentCore Evaluations を使用してエージェントの動作を評価できます。オンライン評価では、追加の作業は必要ありません。ライブダッシュボードからエージェントのパフォーマンスを監視するだけです。

### オンライン評価がトレースとどのように連携するか

オンライン評価では、エージェントが呼び出されて AgentCore Observability にトレースが生成されます。これらのトレースはセッションにマッピングされ、ログは Amazon CloudWatch Log グループで利用可能になります。オンライン評価では、開発者が特定のエージェントに対してオンライン評価設定を作成し、サンプルレートとこの設定に適用する評価メトリクスを定義します。AgentCore Evaluations は、設定されたサンプリングレートに従って生成されたトレースを分析し、本番環境でエージェントを自動的に評価します。その後、開発者は AgentCore Observability ダッシュボードを使用してエージェントからのトレースと評価スコアを可視化し、評価結果に基づいてエージェントを継続的に更新できます。


![session_traces](../../images/online_evaluations.png)

### 前のチュートリアルからの情報の取得

このチュートリアルでは、前提条件チュートリアルで AgentCore Runtime にデプロイした Strands エージェントを使用します。事前構築されたメトリクスと、`01-creating-custom-metrics` チュートリアルで作成した `response_quality` メトリクスで評価します。エージェントと評価メトリクスの情報を取得しましょう。

In [ ]:
%store -r launch_result_strands
%store -r evaluator_id
try:
    print("Agent Id:", launch_result_strands.agent_id)
    print("Agent ARN:", launch_result_strands.agent_arn)
except NameError as e:
    raise Exception("""Missing launch results from your Strands agent. Please run 00-prereqs before executing this lab""")

try:
    print("Evaluator id:", evaluator_id)
except NameError as e:
    raise Exception("""Missing custom evaluator id. Please run 01-creating-custom-evaluators before executing this lab""")

### AgentCore Evaluations クライアントの初期化

それでは AgentCore Starter toolkit から AgentCore Evaluations クライアントを初期化しましょう。

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Observability
import os
import json
from boto3.session import Session
from IPython.display import Markdown, display

In [ ]:
boto_session = Session()
region = boto_session.region_name
print(region)

In [ ]:
eval_client = Evaluation(region=region)

### オンライン評価設定の構成

それではオンライン評価設定を構成しましょう。この場合、デモ目的でのみエージェントを使用しているため、生成されるすべてのトレースを評価します。実際のアプリケーションでは、エージェントの利用状況に応じてサンプルレートを設定する必要があります。

オンデマンド評価で探索した5つのメトリクスで評価設定を作成します:
* Builtin.GoalSuccessRate
* Builtin.Correctness
* Builtin.ToolParameterAccuracy
* Builtin.ToolSelectionAccuracy および
* カスタムメトリクス: response_Quality

In [ ]:
response = eval_client.create_online_config(
    agent_id=launch_result_strands.agent_id,
    config_name="strands_agent_eval2",
    sampling_rate=100,
    evaluator_list=[
        "Builtin.GoalSuccessRate", "Builtin.Correctness", 
        "Builtin.ToolParameterAccuracy", "Builtin.ToolSelectionAccuracy",
        evaluator_id
    ],
    config_description="Strands agent online evaluation test",
    auto_create_execution_role=True
)

### 評価設定の分析

オンライン評価設定の設定 ID を確認しましょう:

In [ ]:
print("Online Evaluation Configuration Id:", response['onlineEvaluationConfigId'])

作成した設定の詳細を確認して、すでに有効になっていることを確認することもできます:

In [ ]:
eval_client.get_online_config(config_id=response['onlineEvaluationConfigId'])

### エージェントを呼び出して評価をトリガー

オンライン評価をトリガーするために、いくつかの新しいクエリでエージェントを呼び出しましょう。今回はエンドポイントが利用可能になると任意のインターフェースから呼び出せるため、boto3 でエージェントを呼び出します。

In [ ]:
import boto3
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

def invoke_agent_runtime(agent_arn, prompt):
    boto3_response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        qualifier="DEFAULT",
        payload=json.dumps({"prompt": prompt})
    )
    if "text/event-stream" in boto3_response.get("contentType", ""):
        content = []
        for line in boto3_response["response"].iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    print(line)
                    content.append(line)
        display(Markdown("\n".join(content)))
    else:
        try:
            events = []
            for event in boto3_response.get("response", []):
                events.append(event)
        except Exception as e:
            events = [f"Error reading EventStream: {e}"]
        display(Markdown(json.loads(events[0].decode("utf-8"))))
    return boto3_response


In [ ]:
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "How much is 7+9+10*2?"
)

In [ ]:
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "Is it raining?"
)

In [ ]:
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "how much is 20% of 300?"
)

In [ ]:
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "What can you do?"
)

In [ ]:
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "What is the capital of NY State?"
)

### オンライン評価の可視化

エージェントとの十分なインタラクションを作成すると、[AgentCore Observability コンソール](https://console.aws.amazon.com/cloudwatch/home#gen-ai-observability/agent-core/agents)を使用して、オンライン評価設定に従ってエージェントがどのように動作しているかを可視化できます。

エージェントの `DEFAULT` エンドポイントに移動して、現在の評価を確認してください

**重要**: 評価結果がダッシュボードに表示されるまで時間がかかる場合があります。評価ダッシュボードが空の場合は、数分待ってから再度確認してください。

利用可能になると、エージェントのトレースでメトリクスを直接確認できます:
![image.png](../../images/online_evaluations_dashboard.png)

### おめでとうございます！

最初のオンライン評価設定を作成しました！これで AgentCore Evaluations でカスタムメトリクスを作成し、エージェントをオンデマンドおよびオンラインで評価できます！